# Modelo de datos

<img align="left" src="https://drive.google.com/uc?export=view&id=1VVCJJtMq8ObGOq5ITvwkrCf_IWIikIkE">

In [1]:
import pandas as pd
import cufflinks as cf

from IPython.display import display,HTML

In [2]:
cf.set_config_file(sharing='public', theme='ggplot', offline=True)

df_sales = pd.read_csv('dataset/sales.csv', sep=',', encoding='utf-8')
df_sales.head(2)

,id_client,id,product,place_purchase,gross_price,net_price,timestamp
0,f3afea952b2a40109df9ba1145d8376c-104,3908,Z,web,11498,9314,"Thu, 20 Jan 2022 00:00:00 GMT"
1,f3afea952b2a40109df9ba1145d8376c-104,3907,Z,tienda-1,13455,10899,"Sat, 22 Jan 2022 00:00:00 GMT"


In [3]:
df_clients = pd.read_csv('dataset/clients.csv', sep=',', encoding='utf-8')
df_clients.head(2)

,id,view_web
0,f9ac3d5f16cf46a0b000b2cf3dee39df,379
1,f97bf03bd79943a297fbb63194441207,548


## Información derivada

$(net + revenue)\cdot 1.19 = gross $

In [4]:
df_sales['revenue'] = (df_sales['gross_price']/1.19) - df_sales['net_price']
df_sales['days'] = [day[:3] for day in df_sales['timestamp']] 

## Analysis

In [5]:
print(df_sales[['gross_price','net_price','revenue']].describe())
df_sales[['gross_price','net_price','revenue']].iplot(kind='box', title=f'Describe data')

         gross_price      net_price      revenue
count    3909.000000    3909.000000  3909.000000
mean    51729.387823   41901.205168  1568.868633
std     51231.478493   41497.498815  1554.163803
min      8641.000000    7000.000000   261.344538
25%     11719.000000    9493.000000   355.016807
50%     14560.000000   11794.000000   441.294118
75%    110255.000000   89307.000000  3344.260504
max    148135.000000  119990.000000  4493.193277


## Analysis per place and product

In [6]:
df = df_sales.pivot(index='id', columns='place_purchase',values='revenue').fillna(0).sum()
df.iplot(kind='bar', xTitle='Place purchase', yTitle='Revenue', title='Revenue per place', colors='Blue')

In [7]:
data = {
    'X': df_sales[df_sales['product']=='X'].pivot(index='id', columns='product',values='revenue').sum(),
    'Y': df_sales[df_sales['product']=='Y'].pivot(index='id', columns='product',values='revenue').sum(),
    'Z': df_sales[df_sales['product']=='Z'].pivot(index='id', columns='product',values='revenue').sum()
}

df = pd.DataFrame(data).fillna(0).sum()
df.iplot(kind='bar', xTitle='Product', yTitle='Revenue', title='Revenue per product')

In [8]:
df = df_sales.groupby(['place_purchase','product'])['revenue'].sum().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Product', yTitle='Revenue', title='Revenue of place per product')

In [9]:
df = df_sales.groupby(['place_purchase','product'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Place', yTitle='Sales quantity', title='Sales quantity per product on place')

## Analysis per client

In [10]:
MUN_CLIENTS = 10
df = df_sales.groupby(['id_client'])['revenue'].sum().sort_values(ascending=False)[:MUN_CLIENTS]
df.iplot(kind='bar', xTitle='Clients', yTitle='Revenue', title=f'the top {MUN_CLIENTS} highest revenue per client ')

In [11]:
df = df_sales.groupby(['id_client','product'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Clients', yTitle='Sales quantity', title=f'Sales quantity per client')

In [12]:
df = df_sales.groupby(['id_client','place_purchase'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Clients', yTitle='Sales quantity', title=f'Sales quantity in place per client')

## Analysis sales in web

In [13]:
# percentage of how much the customer buys on the web

clients_,p, best_clients = [],[],[]

for client, count_sale in zip(df.index,df['web']):
    view = df_clients[df_clients['id'] == client[:-3]]['view_web']
    if view.empty:
        view = 1
    if count_sale/int(view) > 1:
        clients_.append(client)
        p.append(100)
    else:
        clients_.append(client)
        p.append(round(count_sale/int(view)*100,2))

print(f' %\t| id client')
for client_id,percentage in zip(clients_,p):
    print(f'{percentage}\t| {client_id}')
    
    if percentage == 100:
        best_clients.append(client_id)

 %	| id client
4.91	| 0099159dc1c5406c9de9755c9d2d7d61-29
0.62	| 058b4f158bb34a1eb75074270095aae6-73
0.72	| 05be4424291c45f2a201351a109f9a49-61
3.07	| 0924f8f6a81c4ce89c0e3c9f6db4493d-17
2.98	| 09264a55348b4be1ab7387fb079667e8-25
0.2	| 09a92c36e10247b3b6d35f639ec9c097-64
18.18	| 09ae174f1a0946f5848693f694e3e3b1-34
6.99	| 0a5e6748e0c541488f097be6c32ce9e4-28
0.22	| 0ce9d99c9eda43c38c95891f79ef7e61-87
0.82	| 13a3ee4be10c43d98af9a7c3ca9f3e40-91
0.43	| 1668a770eff34814b47e70a60def5408-98
0.17	| 1a2fefd3c1494224977eec61647e3655-82
3.8	| 1beb2bbf03614495b8bc9a4d2999b840-27
9.09	| 1f01551b2a1048f096a77618c1520727-36
0.23	| 2447983cd44847349496238a7a8e95d8-97
0.39	| 2602c52c04694026a697f65bac066835-60
0.19	| 2670d09cef244dd3b9c36501115b6800-71
100	| 2a073a63c6e046198924a6776e12413e-0
4.21	| 305228f683c94543b44254902faf34cd-20
40.0	| 3067409d23d04097b842c3e02112b0a3-35
0.19	| 31f604bb71f64961b382b1ff52b3e771-92
100	| 3828feb3b95c4f9a8bb771c976d286a7-1
3.68	| 394978f0cf62432a8b0361137290d68b-19
2

In [14]:
df_best_clients = df_sales[df_sales['id_client'].isin(best_clients)]
df = df_best_clients.groupby(['id_client','product'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Clients', yTitle='Sales quantity', title=f'customers who compare every time they log on to the website')

In [15]:
df = df_best_clients.groupby(['id_client','days'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Clients', yTitle='Sales quantity', title=f'customers who compare every time they log on to the website')

In [16]:
df =  df_best_clients.groupby(['product','days'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Product', yTitle='Sales quantity', title=f'products sold during the week')

## Analysis sales in time

In [17]:
df = df_sales.groupby(['product','days'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Product', yTitle='Sales quantity', title=f'products sold during the week')

In [18]:
df = df_sales.groupby(['place_purchase','product','days'])['revenue'].count().unstack(fill_value=0)
df.iplot(kind='bar', xTitle='Product', yTitle='Sales quantity', title=f'products sold during the week')